# Interactive visualisation

### This notebook is seventh in the series of soiling detection pipeline notebooks
Dataframes imported are generated in 1_preprocessing, 2_detect_timewindown, 3_big_drop_filter and 4_bad_day_filter notebooks <br>
To analyse other data, CBs needs to be changed to str if the park has strings not CBs

Author: Lisa Crowther

### An interactive plot:
The step of filtering is chosen, and either all strings/CBs or strings/CBs to a particular inverter

In [1]:
import math

import pandas as pd
import numpy as np

from IPython.display import display  # For showing frames

import panel as pn
pn.extension('vega')

import matplotlib.pyplot as plt

#for dashboard
from bokeh.plotting import figure, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_8
from bokeh.models import Legend, LegendItem
from bokeh.palettes import Category20
import seaborn as sns

from bokeh.models import DatetimeTickFormatter

In [2]:

EPI_stringfilt_path ="../data/temp/park2/time_mask_filter/EPI_stringfilt.csv"
EPI_timemask_path = "../data/temp/park2/time_mask_filter/EPI_timemask.csv"

EPI_bigdrop_filt_path = "../data/temp/park2/big_drop_filter/EPI_bigdrop_filt.csv"

EPI_unfilt_path = "../data/temp/park2/preprocessing/df_EPI.csv"
EPI_bigdrop_badday_filt_path= '../data/temp/park2/bad_day_filter/EPI_dropsfilt_baddayfilt.csv'

In [3]:
list_of_filepaths=[EPI_stringfilt_path,EPI_timemask_path,EPI_bigdrop_filt_path,EPI_unfilt_path,EPI_bigdrop_badday_filt_path]

In [4]:
EPI_stringfilt, EPI_timemask,EPI_bigdrop_filt, EPI_unfilt,EPI_bigdrop_badday_filt  = [pd.read_csv(filepath, parse_dates=['datetime'], date_parser = pd.to_datetime, index_col='datetime') for filepath in list_of_filepaths]

In [5]:
EPI_stringfilt_D=EPI_stringfilt.resample(rule='D').median()
EPI_timemask_D=EPI_timemask.resample(rule='D').median()
EPI_unfilt_D=EPI_unfilt.resample(rule='D').median()


In [6]:

EPI_stringfilt_D['med_CBs']=EPI_stringfilt_D.median(axis=1)
EPI_timemask_D['med_CBs']=EPI_timemask_D.median(axis=1)
EPI_bigdrop_filt['med_CBs']=EPI_bigdrop_filt.median(axis=1)
EPI_unfilt_D['med_CBs']=EPI_unfilt_D.median(axis=1)
EPI_bigdrop_badday_filt['med_CBs']=EPI_bigdrop_badday_filt.median(axis=1)

/Users/lisacrowther/opt/anaconda3/envs/VIZ/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [10]:
### HAVE TO CHANGE 'STR' TO 'CB' FOR PARK2 AND 'CB' TO 'STR' FOR PARK1

inverter_group=[1,2,3,4, 'all']

inverter_num =pn.widgets.Select(name='inverter_CBs', options=inverter_group)

#group_class = pn.widgets.Select(name='cluster', options=ranked_group)
# could be filtered vs unfiltered instead of cluster num
filtering = ['EPI_unfilt','EPI_timemask', 'EPI_bigdrop_filt','EPI_bigdrop_badday_filt']

filtering = pn.widgets.Select(name='filtering', options=filtering)

#filtered = pn.widgets.RadioButtonGroup(name='Filtered', options= ['True','False'], button_type ='success')
#need to change df so filtered option is chosen here


#@pn.depends(cluster_num.param.value, parameter.param.value)
@pn.depends(inverter_num.param.value, filtering.param.value)
#def get_plot_filters(group_class, parameter): 
def get_plot_filters(inverter_num,filtering): 
    if filtering =='EPI_unfilt':
        df=EPI_unfilt_D
        title_text= 'unfiltered'
    if filtering =='EPI_timemask':
        df=EPI_timemask_D
        title_text= 'time-mask filtered'
    if filtering =='EPI_bigdrop_filt':
        df=EPI_bigdrop_filt
        title_text= 'big-drop filtered'
    if filtering =='EPI_bigdrop_badday_filt':
        df=EPI_bigdrop_badday_filt 
        title_text= 'bad-day filtered'
    title=f'Daily {title_text} EPI: CBs to inverter {str(inverter_num)} vs median EPI of all CBs' 
    df['Datetime']=pd.to_datetime(df.index)
    colnames=df.columns[:len(df.columns)-2]
    if inverter_num=='all':
        string_names=colnames
        title=f'Daily EPI: all {title_text} CBs vs median EPI of all CBs' 
    else:
        string_names =  colnames[colnames.str.contains('CB '+str(inverter_num))]
    new=df[string_names]
    med=df['med_CBs']
    timecol=df['Datetime']
    new=pd.concat([new, med], axis=1)
    new=pd.concat([new, timecol], axis=1)
    cds = ColumnDataSource(new)
    #group2['Datetime']=group2.index.factorize()[0]
    hover = HoverTool(
            tooltips=[
                ("group_class", "$group_class"),
                ('date','@datetime')
        ]
    )
    
    p = figure(plot_width=800, plot_height=400, tools=[hover], y_range=(0, 1.3))
    p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
    for name in range(len(string_names)):
        p.line('Datetime', string_names[name], source=cds)
    p.line('Datetime','med_CBs',source=cds,  color='red')
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'EPI'
    #p.x_range = Range1d(1, 8, bounds=(1, 8))
    #p.y_range = Range1d(10, 50, bounds=(10, 50))
    p.title.text = title
    p.title.align = "center"
    p.title.text_font_size = "25px"
    return p


In [11]:
newObj=pn.Column('test', 
pn.Row(pn.Column(inverter_num, filtering), 
                         get_plot_filters))


In [12]:
newObj

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Select(name='inverter_CBs', options=[1, 2, 3, 4, 'all'], value=1)
            [1] Select(name='filtering', options=['EPI_unfilt', ...], value='EPI_unfilt')
        [1] ParamFunction(function)